In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/02 21:59:08 WARN Utils: Your hostname, Matheuss-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.192 instead (on interface en0)
26/02/02 21:59:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 21:59:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from functools import reduce
from pathlib import Path
from pyspark.sql import functions as F

def consolidate_year(
    spark: SparkSession,
    base_folder: Path,
    entity: str,
    year: str,
    acronyms: list
):
    """
    Consolidates all acronyms for a given year into one DataFrame.
    """

    dfs = []

    for acronym in acronyms:
        path = base_folder / entity / year / acronym

        if not path.exists():
            print(f"[WARNING] Path not found: {path}")
            continue

        df = (
            spark.read
                .format("csv")
                .option("header", "true")
                .option("sep", ";")
                .option("inferSchema", "true")
                .option("encoding", "ISO-8859-1")
                .load(str(path))
        )

        # add metadata columns
        df = (
            df
            .withColumn("YEAR", F.lit(year))
            .withColumn("ACRONYM", F.lit(acronym))
            .withColumn("ENTITY", F.lit(entity))
        )

        dfs.append(df)

    if not dfs:
        return None

    return reduce(
        lambda a, b: a.unionByName(b, allowMissingColumns=True),
        dfs
    )


In [9]:
def consolidate_all_years(
    spark: SparkSession,
    base_folder: Path,
    entity: str,
    years: list,
    acronyms: list
):
    """
    Consolidates all years and acronyms into a single DataFrame.
    """

    dfs = []

    for year in years:
        df = consolidate_year(
            spark, base_folder, entity, year, acronyms
        )

        if df is not None:
            dfs.append(df)

    if not dfs:
        return None

    return reduce(
        lambda a, b: a.unionByName(b, allowMissingColumns=True),
        dfs
    )


In [10]:
print("YEARS:", YEARS)
print("ACRONYMS:", ACRONYMS)

for year in YEARS:
    for acronym in ACRONYMS:
        path = BASE_FOLDER / ENTITY / year / acronym
        print(path, "exists:", path.exists())


YEARS: ['2020', '2021', '2022', '2023', '2024', '2025']
ACRONYMS: ['BPA', 'DFC', 'DFP', 'DMPL', 'DRE']
cvm_data/CIA_ABERTA/2020/BPA exists: True
cvm_data/CIA_ABERTA/2020/DFC exists: True
cvm_data/CIA_ABERTA/2020/DFP exists: True
cvm_data/CIA_ABERTA/2020/DMPL exists: True
cvm_data/CIA_ABERTA/2020/DRE exists: True
cvm_data/CIA_ABERTA/2021/BPA exists: True
cvm_data/CIA_ABERTA/2021/DFC exists: True
cvm_data/CIA_ABERTA/2021/DFP exists: True
cvm_data/CIA_ABERTA/2021/DMPL exists: True
cvm_data/CIA_ABERTA/2021/DRE exists: True
cvm_data/CIA_ABERTA/2022/BPA exists: True
cvm_data/CIA_ABERTA/2022/DFC exists: True
cvm_data/CIA_ABERTA/2022/DFP exists: True
cvm_data/CIA_ABERTA/2022/DMPL exists: True
cvm_data/CIA_ABERTA/2022/DRE exists: True
cvm_data/CIA_ABERTA/2023/BPA exists: True
cvm_data/CIA_ABERTA/2023/DFC exists: True
cvm_data/CIA_ABERTA/2023/DFP exists: True
cvm_data/CIA_ABERTA/2023/DMPL exists: True
cvm_data/CIA_ABERTA/2023/DRE exists: True
cvm_data/CIA_ABERTA/2024/BPA exists: True
cvm_data/CI

In [ ]:
df_consolidated = consolidate_all_years(
    spark=spark,
    base_folder=BASE_FOLDER,
    entity=ENTITY,
    years=YEARS,
    acronyms=ACRONYMS
)

if df_consolidated is None:
    raise RuntimeError("No data consolidated")

# df_consolidated.printSchema()
df_consolidated.show(5)


root
 |-- CNPJ_CIA: string (nullable = true)
 |-- DT_REFER: date (nullable = true)
 |-- VERSAO: long (nullable = true)
 |-- DENOM_CIA: string (nullable = true)
 |-- CD_CVM: long (nullable = true)
 |-- GRUPO_DFP: string (nullable = true)
 |-- MOEDA: string (nullable = true)
 |-- ESCALA_MOEDA: string (nullable = true)
 |-- ORDEM_EXERC: string (nullable = true)
 |-- DT_FIM_EXERC: date (nullable = true)
 |-- CD_CONTA: string (nullable = true)
 |-- DS_CONTA: string (nullable = true)
 |-- VL_CONTA: double (nullable = true)
 |-- ST_CONTA_FIXA: string (nullable = true)
 |-- YEAR: string (nullable = false)
 |-- ACRONYM: string (nullable = false)
 |-- ENTITY: string (nullable = false)
 |-- DT_INI_EXERC: date (nullable = true)
 |-- COLUNA_DF: string (nullable = true)

+------------------+----------+------+---------------+------+--------------------+-----+------------+-----------+------------+--------+--------------------+-------------+-------------+----+-------+----------+------------+---------+


In [13]:
df_consolidated.filter(F.col("YEAR").isin(["2020", "2021", "2022"])).groupBy("YEAR", "ACRONYM").count().show()

df_consolidated.filter(F.col("YEAR").isin(["2023", "2024", "2025"])).groupBy("YEAR", "ACRONYM").count().show()

+----+-------+------+
|YEAR|ACRONYM| count|
+----+-------+------+
|2020|    BPA|151251|
|2020|    DFC|105798|
|2020|    DFP|374848|
|2020|   DMPL|468322|
|2020|    DRE| 78317|
|2021|    BPA|159122|
|2021|    DFC|112140|
|2021|    DFP|393622|
|2021|   DMPL|491111|
|2021|    DRE| 81659|
|2022|    BPA|160863|
|2022|    DFC|118551|
|2022|    DFP|397255|
|2022|   DMPL|493399|
|2022|    DRE| 82016|
+----+-------+------+



+----+-------+------+
|YEAR|ACRONYM| count|
+----+-------+------+
|2023|    BPA|164123|
|2023|    DFC|122626|
|2023|    DFP|403661|
|2023|   DMPL|508405|
|2023|    DRE| 82972|
|2024|    BPA|154379|
|2024|    DFC|118565|
|2024|    DFP|379401|
|2024|   DMPL|488375|
|2024|    DRE| 77416|
|2025|    BPA|  2514|
|2025|    DFC|  2008|
|2025|    DFP|  6037|
|2025|   DMPL|  7838|
|2025|    DRE|  1150|
+----+-------+------+

